# Trading Crop Commodities Based on Extreme Weather Events During Critical Times of the Year

This presentation guides you through our thought process in attempting to capitalize on extreme weather events that damage the supply of certain crop futures.

## Coffee
Lets start off with coffee. The KC=F is the futures for coffee. 50% of the coffee beans in this futures contract is grown in the mountainous regions of Brazil, specifically, the Varginha region. We will use the code I wrote in coffee_data.py to extract daily temperature highs and lows from this region using NASA's API from the last 10 years. This exports a file called varignha_coffee_temps_10y.csv. Next, after doing some research, we found out that critical growth and harvest seasons for coffee occur at September, October, June, July, and August. We will watch for extreme hot and cold temperatures during these critical periods using the code below.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

coffee_df = pd.read_csv(
    "crops_data/varginha_coffee_temps_10y.csv", index_col="Date", parse_dates=True
)

coffee_prices = yf.download(
    "KC=F", start="2015-01-01", end="2025-11-24", auto_adjust=True
)

ModuleNotFoundError: No module named 'pandas'